# Algebra tricks

This notebook was initially prompted by the question of how zero correlated variables generate the same estimates in individual linear regressions and multivariate regressions. The setup is the following :

$$
\begin{align*}
Y &= \beta_0 + \beta_x X + U \\ \
Y &= \beta_0 + \beta_z Z + U \\
Y &= \beta_0 + \beta_x X + \beta_z Z + U \\
\end{align*}
$$

where

$$
\begin{align*}
E[U|X], E[U|Z], E[U|X,Z] = 0
\end{align*}
$$

If $X$ and $Z$ have a correlation of zero, we expect that $\beta_x$ and $\beta_z$ are the same in the simple linear correlation and in the regression with two dependent variables.

The question now becomes, how do you we ensure zero correlation between pseudo-random vectors? 
Samples generated from random number generators are not typically zero correlated unless we have very large samples. So how could we ensure a specific statistic for our vectors?

Let us generate small vectors and compute their correlation.

In [144]:
# How to use numpy.random here : https://numpy.org/doc/stable/reference/random/index.html#quick-start

import numpy as np
from numpy.random import default_rng
rng = default_rng()

# Data manipulation and I/O

import pandas as pd

# Proper types

from typing import Tuple

You can run the code down below multiple times, you will get a different correlation (on the second diagonal) every time, yet it's far from zero.

In [145]:
n = 20
a = rng.standard_normal(n)
b = rng.standard_normal(n)

np.corrcoef(a,b)

array([[ 1.       , -0.1454901],
       [-0.1454901,  1.       ]])

That's no good because we can't enforce zero correlation like that.

What's interesting though is that we can use some linear algebra to create zero correlated vectors, the math and logic behind it is outlined and explained in details in [Stachurski's Economic Theory](https://python.quantecon.org/_static/lecture_specific/linear_algebra/course_notes.pdf), Chapter 3. Overall, the [Quantecon](https://python.quantecon.org/linear_algebra.html) lectures are excellent. 

In substance, it explains that $\hat{Y} = \hat{\beta}' X$, the estimated values of the dependent variable, is the linear mapping of $Y$ onto $X$. Such that $Y - \hat{Y}$ is orthogonal (shown as $\perp$) to $X$. We can use that property to create uncorrelated samples.

The first function, `create_shape_arrays` let's up setup random vectors `x` and `y` as well as a matrix `X` which will be the input to an OLS regression.

In [146]:
def create_shape_arrays(N: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    x, y = rng.standard_normal(size = (2, N))
    x, y = map(lambda x: x.T, np.atleast_2d(x,y))
    X = np.column_stack((np.ones(N), x))
    return X, y, x

The second function, `ols`, runs the classical OLS regression and outputs the estimated value $\hat{y}$ as well as the residuals of the regression.

In [147]:
def ols(X: np.ndarray ,y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    betas = np.linalg.lstsq(X, y, rcond=None)[0]
    yhat = np.einsum("nk, km -> nm", X, betas)
    resid = y - yhat
    return yhat, resid

`corr_two_vars` is simply a wrapper around `np.corrcoef` to output the value of the correlation rather than the whole correlation matrix.

In [148]:
def corr_two_vars(a: np.ndarray, b: np.ndarray) -> float:
    return np.corrcoef(a,b)[0, 1]

Finally, `ortho_arrays` takes in all we've done before and returns the vector `x` as well as the residuals ($y - \hat{y}$), those are the two we are interested in because there are uncorrelated. It also outputs the correlation between the two vectors.

In [149]:
def ortho_arrays(N: int) -> Tuple[np.ndarray, np.ndarray, float]:
    X, y, x = create_shape_arrays(N)
    yhat, resid = ols(X, y)
    corr = corr_two_vars(x.flatten(), resid.flatten())
    return x, resid, corr

Now we can just run it with an arbitrary small `N` > 1.

In [150]:
N = 20

arr, orthogonal_arr, correlation = ortho_arrays(N)

In [151]:
df = pd.DataFrame(np.column_stack((arr, orthogonal_arr)), columns=["array", "orthgonal array"])

df.head()

,array,orthgonal array
0,2.362291,-0.040094
1,-1.881089,1.452704
2,0.525728,-1.312826
3,-0.384927,-1.402783
4,1.052469,1.007044


In [152]:
# Correlation matrix of the two vectors

df.corr()

,array,orthgonal array
array,1.000000e+00,2.860517e-17
orthgonal array,2.860517e-17,1.000000e+00


Now we can verify exactly that the coefficients are the same for the regressions. We will run three regressions.

In [153]:
# Regression formulas and pretty outputs

import statsmodels.formula.api as smf

In [154]:
N = 30

y = rng.standard_normal(N)
x, z, _ = ortho_arrays(N)

matrix = pd.DataFrame({"y": y, "x": x.flatten(), "z" :z.flatten()})

In [155]:
reg_y_x = smf.ols("y ~ x", data=matrix).fit().params
reg_y_z = smf.ols("y ~ z", data=matrix).fit().params
reg_y_xz = smf.ols("y ~ x + z", data=matrix).fit().params

In [160]:
col_names = {0:"y ~ x", 1:"y ~ z", 2:"y ~ x + z"}

results = pd.DataFrame([reg_y_x, reg_y_z, reg_y_xz]).T

results = results.rename(columns=col_names).replace(np.nan, "")

results

,y ~ x,y ~ z,y ~ x + z
Intercept,-0.191517,-0.198375,-0.191517
x,0.0255694,,0.025569
z,,-0.353022,-0.353022


We can now see that the coefficients are indeed exactly the same!

This is the end of the notebook and the exercise.

### Bonus

Actually, there is a function in scipy that uses Singular Value Decomposition (SVD) to create orthogonal columns from a matrix, it is shown down below. I simply rescaled the matrix to ensure reliable results. 

In [158]:
from scipy.linalg import orth

def uncorrelated_matrix(size=None):
    X = rng.standard_normal(size)
    X = orth(X - X.mean(axis=0))
    X = X/X.std(axis=0)
    return X

mat = uncorrelated_matrix(size = (30, 4))

print("\n","Correlations of the columns", "\n")
pd.DataFrame(np.corrcoef(mat, rowvar=False), columns=[f"column {i}" for i in range(1, mat.shape[1]+1)])


 Correlations of the columns 



,column 1,column 2,column 3,column 4
0,1.000000e+00,1.117515e-17,-8.205902e-18,2.256900e-16
1,1.117515e-17,1.000000e+00,-1.725056e-16,1.623990e-16
2,-8.205902e-18,-1.725056e-16,1.000000e+00,3.447038e-16
3,2.256900e-16,1.623990e-16,3.447038e-16,1.000000e+00


END